In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
dataset = dataset.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)

In [6]:
# encode categorical variable
label_encoder_gender = LabelEncoder()
dataset['Gender'] = label_encoder_gender.fit_transform(dataset['Gender'])


In [7]:
# One Hot Encoder for Geography
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoder = onehot_encoder_geo.fit_transform(dataset[['Geography']]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder, columns = onehot_encoder_geo.get_feature_names_out(['Geography']))

In [8]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
dataset = pd.concat([dataset.drop('Geography', axis = 1), geo_encoder_df], axis = 1)
dataset

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [12]:
# split dataset into dependent and independent datasets
X = dataset.drop('EstimatedSalary', axis = 1)
y =dataset['EstimatedSalary']

In [13]:
## load the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo = pickle.load(file) 

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)    

In [14]:
# split train test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [15]:
# scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
## ANN Regression problem statement
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
model = Sequential(
    [
        Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)), # first hidden layer connected to input layer
        Dense(32, activation  ='relu'), # Hidden layer 2
        Dense(1) # output layer
    ]
)

In [18]:
model.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mae'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = 'regressionlogs/fit' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)

In [21]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, callbacks = [tensorflow_callback, early_stopping_callback])

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 100371.8125 - mae: 100371.8125 - val_loss: 98496.2188 - val_mae: 98496.2188
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 99557.0000 - mae: 99557.0000 - val_loss: 96845.8125 - val_mae: 96845.8125
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 96700.6406 - mae: 96700.6406 - val_loss: 92651.1406 - val_mae: 92651.1406
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 91135.9766 - mae: 91135.9766 - val_loss: 85749.4062 - val_mae: 85749.4062
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 83125.9844 - mae: 83125.9844 - val_loss: 76935.6094 - val_mae: 76935.6094
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 73834.2734 - mae: 73834.2734 - val_loss: 67939.6484 - val_mae: 67939.6484
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 65006.6484 - mae: 65006.6484 

In [22]:
%load_ext tensorboard

In [25]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 38535), started 0:00:59 ago. (Use '!kill 38535' to kill it.)

In [26]:
# Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)
test_mae

63/63 [==============================] - 0s 1ms/step - loss: 50233.1172 - mae: 50233.1172


50233.1171875

In [27]:
model.save('regression_model.h5')

/Users/siddharthgehlot/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
